In [1]:
# using Pkg; Pkg.add("Distributions")

In [2]:
using Distributions
using DataFrames
using Distributed

In [3]:
addprocs(8)

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [4]:
nworkers()

8

In [5]:
using Distributed
@everywhere using Distributed, Distributions, DataFrames, Random

In [6]:
@everywhere function sim_inventory(reorder_q::Int64,
                                   reorder_point::Int64;
        days = 100,
        sd = Normal(20,20^0.5),     # 日毎の売上の分布
        wh = 0.1,                   #  在庫コスト
        p = 4.0,                    #  ユニットあたりの売上利益
        d_prob = 0.50,              #  日毎の注文分布
        k = 60.0,                   # 固定配送コスト
        rng = MersenneTwister(0))
    profit = 0.0                    # 利益の総計
    stock = reorder_q
    for day in 1:days
        if stock < reorder_point && rand(rng)< d_prob # 注文到着
            profit -= k                #  配送料支払い
            stock += reorder_q
        end
        sale = max(0, min(Int(round(rand(rng, sd))), stock))
        stock -= sale                #  在庫を減らす
        profit += p*sale - wh*stock #  利益を増やす
    end
    return profit / days
end

In [7]:
sweep = vec(collect(Base.product(10:10:250, 10:10:250, 20:5:60)))

5625-element Array{Tuple{Int64,Int64,Int64},1}:
 (10, 10, 20)  
 (20, 10, 20)  
 (30, 10, 20)  
 (40, 10, 20)  
 (50, 10, 20)  
 (60, 10, 20)  
 (70, 10, 20)  
 (80, 10, 20)  
 (90, 10, 20)  
 (100, 10, 20) 
 (110, 10, 20) 
 (120, 10, 20) 
 (130, 10, 20) 
 ⋮             
 (140, 250, 60)
 (150, 250, 60)
 (160, 250, 60)
 (170, 250, 60)
 (180, 250, 60)
 (190, 250, 60)
 (200, 250, 60)
 (210, 250, 60)
 (220, 250, 60)
 (230, 250, 60)
 (240, 250, 60)
 (250, 250, 60)

In [8]:
Random.seed!(0);
Random.shuffle!(sweep);

In [9]:
const rngs = Dict(i=>MersenneTwister(i) for i in workers());

In [10]:
res = @distributed (append!) for s in sweep
    rng = deepcopy(rngs[myid()])
    profit = 0.0
    for sim in 1:10000
        profit += sim_inventory(s[1], s[2], days=s[3], rng=rng)
    end
    DataFrame(worker=myid(), reorder_q=s[1], reorder_point=s[2],
           days=s[3], profit=profit/10000)
end

,worker,reorder_q,reorder_point,days,profit
,Int64,Int64,Int64,Int64,Float64
1,2,160,30,35,61.3314
2,2,190,160,25,51.4862
3,2,120,170,35,50.337
4,2,210,120,30,55.7457
5,2,40,10,40,33.3784
6,2,210,150,55,53.0972
7,2,140,60,20,61.8401
8,2,60,100,20,49.4209
9,2,70,40,55,53.416


In [11]:
DataFrame([sdf[argmax(sdf.profit), :] for sdf in groupby(res, :days, sort=true)])

,worker,reorder_q,reorder_point,days,profit
,Int64,Int64,Int64,Int64,Float64
1,5,210,30,20,63.7809
2,9,170,40,25,62.8526
3,5,210,40,30,62.5125
4,9,180,40,35,62.3673
5,6,200,40,40,61.9569
6,2,180,40,45,61.8942
7,8,170,40,50,61.738
8,9,180,40,55,61.6621
9,7,170,40,60,61.5416
